In [8]:
import os, pandas as pd, numpy as np, joblib, shap, matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

# 1) --- Load new featured + clustered data ---
df = pd.read_csv('CSV_files/final_clustered_Test_set.csv')

# 2) --- Exact feature list used during training ---
exclude = ['deviceID','tripID','cluster','label',
           'anomaly_score_ebm','alert_ebm',
           'anomaly_score_iforest','alert_iforest',
           'y_pred','residual']
feature_cols = [c for c in df.columns if c not in exclude]

# 3) --- Ensure predictions exist (per-cluster EBM inference) ---
if 'y_pred' not in df.columns:
    df['y_pred'] = np.nan
    for c in sorted(df['cluster'].unique()):
        model = joblib.load(f'models/ebm_regressor_cluster_{int(c)}.joblib')
        m = df['cluster'] == c
        Xc = df.loc[m, feature_cols]
        df.loc[m, 'y_pred'] = model.predict(Xc)

# --- Compute anomaly_score_ebm & alert_ebm if missing ---
if 'anomaly_score_ebm' not in df.columns:
    if 'kpl_mean' in df.columns and 'y_pred' in df.columns:
        df['anomaly_score_ebm'] = (df['kpl_mean'] - df['y_pred']).abs()
    else:
        raise ValueError("Cannot compute anomaly_score_ebm: missing 'kpl_mean' or 'y_pred'")

if 'alert_ebm' not in df.columns:
    threshold = np.percentile(df['anomaly_score_ebm'], 95)  # Top 5% anomalies
    df['alert_ebm'] = df['anomaly_score_ebm'] >= threshold

# --- Create folders for explanations ---
os.makedirs('explanations_new/per_row', exist_ok=True)
os.makedirs('explanations_new/summary', exist_ok=True)

# 4) --- SHAP Explanations (per cluster) ---
for c in sorted(df['cluster'].unique()):
    m_cluster = df['cluster'] == c
    if m_cluster.sum() == 0:
        continue

    model = joblib.load(f'models/ebm_regressor_cluster_{int(c)}.joblib')

    X_bg = df.loc[m_cluster, feature_cols].sample(
        n=min(200, m_cluster.sum()), random_state=42
    ).copy()[feature_cols]

    predict_fn = lambda X: model.predict(pd.DataFrame(X, columns=feature_cols))
    explainer = shap.Explainer(predict_fn, X_bg)

    # Global summary
    Xc = df.loc[m_cluster, feature_cols]
    sv = explainer(Xc)
    shap.summary_plot(sv, Xc, show=False)
    plt.savefig(f'explanations_new/summary/shap_summary_cluster_{int(c)}.png', dpi=160, bbox_inches='tight')
    plt.close()

    # Per-row explanations (alerts first, else top residuals)
    MAX_ROWS = 20
    sub = df[m_cluster].copy()
    sub = sub[sub['alert_ebm'] == True]

    if sub.empty:
        if 'residual' not in df.columns:
            df.loc[m_cluster, 'residual'] = (df.loc[m_cluster, 'kpl_mean'] - df.loc[m_cluster, 'y_pred']).abs()
        sub = df.loc[m_cluster].sort_values('residual', ascending=False)

    if len(sub) > MAX_ROWS:
        sub = sub.sample(n=MAX_ROWS, random_state=42)

        for idx, r in sub.iterrows():
        # --- FIX: Convert row to clean numeric float array ---
            x_row = r[feature_cols].to_frame().T
            x_row = x_row.apply(pd.to_numeric, errors='coerce')  # force numeric
            x_row = x_row.fillna(X_bg.mean())                    # fill NaNs with background mean
            x_row = x_row.astype(float)                          # ensure float dtype

            sv_row = explainer(x_row)
            base = float(sv_row.base_values[0])
            vals = sv_row.values[0]
            feat = x_row.iloc[0]

            html = shap.force_plot(base, vals, feat, matplotlib=False)
            name = f"force_cluster{int(c)}_dev{r.get('deviceID','NA')}_trip{r.get('tripID','NA')}"
            with open(f'explanations_new/per_row/{name}.html', 'w', encoding='utf-8') as f:
                f.write(shap.getjs() + html.html())

            plt.figure()
            shap.force_plot(base, vals, feat, matplotlib=True, show=False)
            plt.savefig(f'explanations_new/per_row/{name}.png', dpi=200, bbox_inches='tight')
            plt.close()


# 5) --- Isolation Forest Comparison ---
X = df[feature_cols]
iforest = IsolationForest(n_estimators=200, contamination=0.05, random_state=42)
iforest.fit(X)

df['anomaly_score_iforest'] = iforest.decision_function(X) * -1  # flip so higher = worse
df['alert_iforest'] = iforest.predict(X) == -1

# 6) --- Agreement & Breakdown ---
ebm_alerts = df['alert_ebm']
iforest_alerts = df['alert_iforest']

agreement = (ebm_alerts == iforest_alerts).mean() * 100
print(f"\n=== Agreement ===")
print(f"Agreement Rate (EBM vs IForest): {agreement:.2f}%")

both_true = ((ebm_alerts) & (iforest_alerts)).sum()
ebm_only = ((ebm_alerts) & (~iforest_alerts)).sum()
iforest_only = ((~ebm_alerts) & (iforest_alerts)).sum()
none = ((~ebm_alerts) & (~iforest_alerts)).sum()

print(f"Both True: {both_true} | EBM Only: {ebm_only} | IForest Only: {iforest_only} | Neither: {none}")

# 7) --- Histogram of Scores ---
plt.figure(figsize=(10,5))
plt.hist(df['anomaly_score_ebm'], bins=30, alpha=0.5, label='EBM')
plt.hist(df['anomaly_score_iforest'], bins=30, alpha=0.5, label='IForest')
plt.xlabel("Anomaly Score")
plt.ylabel("Frequency")
plt.legend()
plt.title("Distribution of Anomaly Scores: EBM vs IForest")
plt.tight_layout()
plt.savefig("explanations_new/summary/score_hist_comparison.png", dpi=160)
plt.close()

# 8) --- Scatter Plot (Correlation between models) ---
plt.figure(figsize=(6,6))
plt.scatter(df['anomaly_score_ebm'], df['anomaly_score_iforest'], alpha=0.5)
plt.xlabel("EBM Anomaly Score")
plt.ylabel("IForest Anomaly Score")
plt.title("EBM vs IForest Anomaly Score Correlation")
plt.tight_layout()
plt.savefig("explanations_new/summary/score_scatter_comparison.png", dpi=160)
plt.close()

print("\nAnalysis complete. Check 'explanations_new/summary/' for plots.")


PermutationExplainer explainer: 540it [00:44,  9.64it/s]                         
C:\Users\saxen\AppData\Local\Temp\ipykernel_26232\3111848678.py:56: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(sv, Xc, show=False)
PermutationExplainer explainer: 4067it [02:22, 26.37it/s]                          
C:\Users\saxen\AppData\Local\Temp\ipykernel_26232\3111848678.py:56: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(sv, Xc, show=False)



=== Agreement ===
Agreement Rate (EBM vs IForest): 90.49%
Both True: 17 | EBM Only: 224 | IForest Only: 214 | Neither: 4150

Analysis complete. Check 'explanations_new/summary/' for plots.
